Detailed ad retrieval from 3 websites ingatlanok.hu,ingatlan.com, otthoncentrum.

Step 1: retrive unique ad's urls from the search engine result to 'urls' variable for a give district
Step 2: use the urls to retrive the ad homepage <body> tag
Step 3: save the ads into source specific .csv

In [1]:
    #urls
    url1a = 'https://ingatlan.com/szukites/elado+lakas+xi-ker+48-55-m2+ar-szerint?page='
    url1b = 'https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page='
    url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
    url2a = 'https://ingatlanok.hu/elado/lakas+haz/budapest-xi-ker:48nm-tol;55nm-ig?record=0&num=100'
    url2b = 'https://ingatlanok.hu/elado/lakas+haz/budapest-vi-ker:38nm-tol;55nm-ig?record=0&num=100'
    url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    url3a = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest11-kerulet/hely-id:budapest11-kerulet,/netto-alapterulet:45~55/emelet:1,8'
    url3b = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest06-kerulet/hely-id:budapest06-kerulet,/netto-alapterulet:38~55/emelet:1,8'

    url1 = 'https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page='
    url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
    url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    


In [2]:
import requests
#from urllib.robotparser import RobotFileParser
from bs4 import BeautifulSoup
import urllib3
import warnings
warnings.filterwarnings('ignore')
import datetime as dtime
import time
import pandas as pd

In [3]:
#searchpage retrival

In [4]:
#ingatlan.com, multi-page
def inner_multi_get(url, n):
    '''
    gets multipages session based webpages like search results specialized to ingatlan.com site
    
    url for URL
    n for number of pages to follow
    
    returns ads specific url in list
    '''
    urls = []
    link = url + str(n)
    print(link)
    http = urllib3.PoolManager(headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    response = http.request('GET', link)
    soup = BeautifulSoup(response.data, 'lxml')
    l = soup.find_all('div', attrs = {'class': 'listing js-listing listing--cluster-parent js-cluster-parent'})
    l = l + (soup.find_all('div', attrs = {'class': 'listing js-listing'}))
    for i in l:
            urls.append('https://ingatlan.com'+i.find('a')['href'])
    
    return urls

def multipagesSearch(url , v=10):
    '''
    wrapper for inner_multi_get(url, n) to introduce timer between page download 
    url for URL,
    v for number of pages to return
    
    inner_multi_get:
    gets multipages session based webpages like search results specialized to ingatlan.com site
    
    returns ad specific url in list
    '''
    i=1
    while i <= v:
        text = inner_multi_get(url, n=i)
        
        if 'csvtext' in locals():
            csvtext = csvtext + text
        else: csvtext = text
            
        time.sleep(1)
        i += 1
    return csvtext

#ingatlanok.hu, oc.hu, one page
def simpleSearch(url):
    '''
    gets simple one page website from the URL
    
    returns ad specific url for .hu and oc.hu pages
    '''
    r=requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    
    page = BeautifulSoup(r.content, 'lxml')
    body = page.body
    url_list = []
    if 'ingatlanok.hu' in url:
        a = body.find_all('div', attrs={'class': 'result-row'})
        for i in a:
            url_list.append(i.find_all('a')[1]['href'])
    if 'oc.hu' in url:
        a = body.find_all('div', attrs={'class': 'estate-list-box'})
        for i in a:
            url_list.append('https://www.oc.hu'+i.find_all('a')[0]['href'])
    return url_list

In [5]:
#compose for searPage url retrieval
def searchPage(district):
    '''
    retrives the search page results ad's url into a [list] according to the given district.
    Options:
    District: 'V', 'VI','XI'
    '''
    if district == 'V':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-v-ker:25nm-tol;33nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest05-kerulet/hely-id:budapest05-kerulet,/netto-alapterulet:28~33/emelet:1,8'
    elif district == 'VI':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-vi-ker:38nm-tol;55nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest06-kerulet/hely-id:budapest06-kerulet,/netto-alapterulet:38~55/emelet:1,8'
    elif district == 'XI':
        url1 = 'https://ingatlan.com/szukites/elado+lakas+xi-ker+48-55-m2+ar-szerint?page='
        url2 = 'https://ingatlanok.hu/elado/lakas+haz/budapest-xi-ker:48nm-tol;55nm-ig?record=0&num=100'
        url3 = 'https://www.oc.hu/ingatlanok/lista/oldalszam:48/rendezes:relevance/felhasznalas:elado/jogi-status:hasznalt/tipus:flat/stilus:brick,/hely-ertek:budapest11-kerulet/hely-id:budapest11-kerulet,/netto-alapterulet:45~55/emelet:1,8'
    
    hu = simpleSearch(url2)
    print('Hu search: ',len(hu))
    
    com = multipagesSearch(url1)
    print('Com search: ',len(com))
    
    cen = simpleSearch(url3)
    print('Cen search: ',len(cen))
    
    alls = [*hu,*com,*cen]
    print('All urls: ', len(alls))
    
    alls =[i for i in alls if 'http' in i]
    return alls

In [6]:
# detailed webpage retrival

In [7]:
def simple(url):
    '''
    Returns the page <body> tag from url retrieval
    '''
    page = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:62.0) Gecko/20100101 Firefox/62.0'})
    body = BeautifulSoup(page.content, 'lxml').body

    return body

def details(urls, district):
    '''
    Go through the list of urls, retrieves the body tag. 
    Sorted by url, saves the body tag of those that has the keyword specific to the url source 
    
    district: 'V', 'VI', 'XI'
    '''
    
    adshu = []
    adscom = []
    adscen = []
    n=0
    for i in urls:
        print(n,'\t',i)
        if 'ingatlanok.hu' in i:
            try:
                back = simple(i)               
                txt = back.get_text()

                if 'Azonosító:' in txt:             
                    adshu.append(back)
                    
            except: pass
            
        elif 'oc.hu' in i:
            try:
                back = simple(i)               
                txt = back.get_text()
                
                if 'Regisztrációs szám:' in txt:             
                    adscen.append(back)
            except: pass
            
        else:
            try:
                back = simple(i)               
                txt = back.get_text()
                
                if 'Parkolás' in txt or 'Ingatlan állapota' in txt:             
                    adscom.append(back)
            except: pass
        
        n+=1 # counter
    
    now = dtime.datetime.now()
    m = [str(now.month) if len(str(now.month)) == 2 else '0' + str(now.month)]
    d = [str(now.day) if len(str(now.day)) == 2 else '0' + str(now.day)]
    y = str(now.year)
    
    with open('adshu'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adshu)
    with open('adscom'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adscom)
    with open('adscen'+ district + '_' + d[0] + m[0] + y + '.csv', 'w') as f:
                f.write('%s' % adscen)
    return

In [8]:
def checkDownload(date, d = ['V','VI','XI']):
    '''
    Check the No of saved ads.
    Search by date
    Search by district in list format -> ['XX']
    '''
    for n in d:
        for i in ['adshu'+n,'adscom'+n,'adscen'+n]:
            try:
                f = i+ '_' + date +'.csv'
                with open(f, 'r') as r:
                    l = r.read()
                ll = l.split('</body>,')
                print('Saved ads in {}: {}'.format(i,len(ll)))
            except: print('No such file ' + f)

In [9]:
for i in ['V','VI', 'XI']:
    urls = searchPage(i)
    details(urls,i)

Hu search:  20
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=1
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=2
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=3
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=4
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=5
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=6
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=7
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=8
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=9
https://ingatlan.com/szukites/elado+lakas+v-ker+25-33-m2+ar-szerint?page=10
Com search:  47
Cen search:  2
All urls:  69
0 	 https://ingatlanok.hu/elado/lakas/budapest_v_ker/36-millio-ft;32-negyzetmeter;1-szoba;tegla-epitesu/9737873
1 	 https://ingatlanok.hu/elado/lakas/budapest_v_ker/36-millio-ft;32-negyzetm

https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=2
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=3
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=4
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=5
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=6
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=7
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=8
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=9
https://ingatlan.com/szukites/elado+lakas+vi-ker+38-55-m2+ar-szerint?page=10
Com search:  200
Cen search:  48
All urls:  348
0 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/56-millio-ft;53-negyzetmeter;2-szoba;tegla-epitesu/9945260
1 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/39-millio-ft;46-negyzetmeter;1-szoba;tegla-epitesu/9945261
2 	 https://ingatlanok.hu/elado/lakas/bud

65 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/43-millio-ft;49-negyzetmeter;2-szoba;tegla-epitesu/9919312
66 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/34-millio-ft;41-negyzetmeter;1-szoba;tegla-epitesu/9925899
67 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/41-millio-ft;39-negyzetmeter;1-szoba;tegla-epitesu/9951554
68 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/32-millio-ft;39-negyzetmeter;1-1-szoba;tegla-epitesu/9910585
69 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/48-millio-ft;45-negyzetmeter;2-szoba;tegla-epitesu/9901595
70 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/36-millio-ft;51-negyzetmeter;1-szoba;tegla-epitesu/9910185
71 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/49-millio-ft;47-negyzetmeter;2-szoba;tegla-epitesu/9725543
72 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/41-millio-ft;44-negyzetmeter;2-szoba;tegla-epitesu/9855716
73 	 https://ingatlanok.hu/elado/lakas/budapest_vi_ker/39-millio-ft;48-negyzetmeter;3-

140 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+lovolde+ter/29663924
141 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+terez+korut/30684328
142 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+terez+korut/30549742
143 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+podmaniczky+utca/31151452
144 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/31214128
145 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/30270596
146 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/31281052
147 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+podmaniczky+utca/31169131
148 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+podmaniczky+utca/31046989
149 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/b

220 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/30531100
221 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/30549496
222 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/30468628
223 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+bajza+utca/30446605
224 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+bajza+utca/30802876
225 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+lehel+utca+5/29808434
226 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+felso+erdosor/31163779
227 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet+eotvos+utca/31350850
228 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/30391378
229 	 https://ingatlan.com/vi-ker/elado+lakas/tegla-epitesu-lakas/budapest+6+kerulet/31230232

300 	 https://www.oc.hu/ingatlanok/H404024?c=79cc83b7
301 	 https://www.oc.hu/ingatlanok/H404412?c=79cc83b7
302 	 https://www.oc.hu/ingatlanok/H405053?c=79cc83b7
303 	 https://www.oc.hu/ingatlanok/H405525?c=79cc83b7
304 	 https://www.oc.hu/ingatlanok/LK491591?c=79cc83b7
305 	 https://www.oc.hu/ingatlanok/LK478521?c=79cc83b7
306 	 https://www.oc.hu/ingatlanok/LK500476?c=79cc83b7
307 	 https://www.oc.hu/ingatlanok/LK499843?c=79cc83b7
308 	 https://www.oc.hu/ingatlanok/H404458?c=79cc83b7
309 	 https://www.oc.hu/ingatlanok/H404506?c=79cc83b7
310 	 https://www.oc.hu/ingatlanok/H404677?c=79cc83b7
311 	 https://www.oc.hu/ingatlanok/H404318?c=79cc83b7
312 	 https://www.oc.hu/ingatlanok/H404841?c=79cc83b7
313 	 https://www.oc.hu/ingatlanok/H404885?c=79cc83b7
314 	 https://www.oc.hu/ingatlanok/H401438?c=79cc83b7
315 	 https://www.oc.hu/ingatlanok/H401440?c=79cc83b7
316 	 https://www.oc.hu/ingatlanok/H397999?c=79cc83b7
317 	 https://www.oc.hu/ingatlanok/H397869?c=79cc83b7
318 	 https://www.oc.hu/

41 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/52-millio-ft;50-negyzetmeter;2-szoba;tegla-epitesu/9934982
42 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/kelenfold;32-millio-ft;50-negyzetmeter;2-szoba;panel-epitesu/9547732
43 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/34-millio-ft;54-negyzetmeter;2-szoba;panel-epitesu/9901488
44 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/39-millio-ft;49-negyzetmeter;1-szoba;tegla-epitesu/9876243
45 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/35-millio-ft;50-negyzetmeter;2-szoba;panel-epitesu/9861775
46 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/52-millio-ft;50-negyzetmeter;2-szoba;tegla-epitesu/9935101
47 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/37-millio-ft;50-negyzetmeter;2-szoba;csusztatott-zsalu-epitesu/9902014
48 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/35-millio-ft;50-negyzetmeter;2-szoba;nincs-megadva-epitesu/9951840
49 	 https://ingatlanok.hu/elado/lakas/budapest_xi_ker/39-

115 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+andor+utca/31324801
116 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+etele+ut/31339399
117 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+menyecske+utca/31084642
118 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+etele+ut+56/31253386
119 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+bukkony+utca/31089037
120 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+igmandi+utca+5/29618399
121 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+igmandi+utca+5/31379746
122 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/30877660
123 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fehervari+ut/30543865
124 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fegyvernek+utca/30294791
125 	 https

200 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fegyvernek+utca/31342669
201 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31158214
202 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+safrany+utca/31375084
203 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31296991
204 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31157335
205 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fogocska+utca+9/31280641
206 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fogocska+utca+9/31273102
207 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fogocska+utca+9/31273582
208 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fogocska+utca+9/31273111
209 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+etele+ut/31109329
210 	 https://ingatlan.com/x

284 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+bartok+bela+ut/31220035
285 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+bartok+bela+ut/31215928
286 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31159744
287 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31157326
288 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31154713
289 	 https://ingatlan.com/xi-ker/elado+lakas/tegla-epitesu-lakas/budapest+11+kerulet+bartok+bela+ut+56/31139761
290 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fegyvernek+utca/29900732
291 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet/31390888
292 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+fehervari+ut/31160020
293 	 https://ingatlan.com/xi-ker/elado+lakas/panel-lakas/budapest+11+kerulet+hadak+utja+6/30967729
294 	 https://in

In [10]:
import os
for i in sorted(os.listdir()):
    if 'ads' in i:
        print(i,os.stat(i).st_size/(1024*1024))

adscenVI_03032020.csv 5.181197166442871
adscenVI_08032020.csv 5.192955017089844
adscenVI_16032020.csv 5.227396011352539
adscenV_01032020.csv 0.1992168426513672
adscenV_08032020.csv 0.1992168426513672
adscenV_16032020.csv 0.2008962631225586
adscenV_24022020.csv 0.18407058715820312
adscenXI_04032020.csv 2.9646434783935547
adscenXI_08032020.csv 3.059565544128418
adscenXI_16032020.csv 2.9908761978149414
adscomVI_03032020.csv 17.440503120422363
adscomVI_08032020.csv 17.503594398498535
adscomVI_16032020.csv 17.520760536193848
adscomV_01032020.csv 4.044032096862793
adscomV_08032020.csv 3.8627395629882812
adscomV_16032020.csv 4.136160850524902
adscomV_24022020.csv 4.11383056640625
adscomXI_04032020.csv 17.509052276611328
adscomXI_08032020.csv 17.49264144897461
adscomXI_16032020.csv 17.505870819091797
adshuVI_03032020.csv 5.212062835693359
adshuVI_08032020.csv 5.17128849029541
adshuVI_16032020.csv 5.148438453674316
adshuV_01032020.csv 0.8424806594848633
adshuV_08032020.csv 1.0036993026733398
ad

In [25]:
checkDownload('08032020', ['VI'])

Saved ads in adshuVI: 100
Saved ads in adscomVI: 200
Saved ads in adscenVI: 48
